In [1]:
#!pip install skater

In [2]:
import pandas as pd
from subprocess import call
import seaborn as sns
import numpy as np
from scipy import stats
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import seaborn as sn
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer

import warnings
import random
from datetime import datetime
import time
import pickle
from IPython.display import HTML, display
import statsmodels.api as sa
import statsmodels.formula.api as sfa


warnings.filterwarnings('ignore')

In [3]:
#fonctions utiles
def classification_report_opti(y_test, y_test_predict):
    classification = {}
    for cl in range(max(y_test) + 1):
        tp = 0
        fp = 0
        fn = 0
        tn = 0
        for i in range(len(y_test)):
            if cl > 0 and cl < max(y_test):
                if y_test[i] == cl and y_test_predict[i] in [
                        cl - 1, cl, cl + 1
                ]:
                    tp += 1
                elif y_test[i] == cl and y_test_predict[i] not in [
                        cl - 1, cl, cl + 1
                ]:
                    fn += 1
                elif y_test[i] != cl and y_test_predict[i] in [
                        cl - 1, cl, cl + 1
                ]:
                    fp += 1
                elif y_test[i] != cl and y_test_predict[i] not in [
                        cl - 1, cl, cl + 1
                ]:
                    tn += 1
            elif cl == 0:
                if y_test[i] == cl and y_test_predict[i] in [cl, cl + 1]:
                    tp += 1
                elif y_test[i] == cl and y_test_predict[i] not in [cl, cl + 1]:
                    fn += 1
                elif y_test[i] != cl and y_test_predict[i] in [cl, cl + 1]:
                    fp += 1
                elif y_test[i] != cl and y_test_predict[i] not in [cl, cl + 1]:
                    tn += 1
            elif cl == max(y_test):
                if y_test[i] == cl and y_test_predict[i] in [cl, cl - 1]:
                    tp += 1
                elif y_test[i] == cl and y_test_predict[i] not in [cl, cl - 1]:
                    fn += 1
                elif y_test[i] != cl and y_test_predict[i] in [cl, cl - 1]:
                    fp += 1
                elif y_test[i] != cl and y_test_predict[i] not in [cl, cl - 1]:
                    tn += 1
            classification[cl] = {'TN': tn, 'FN': fn, 'FP': fp, 'TP': tp}

    for i in classification:
        precision = (
            classification[i]['TP'] /
            (classification[i]['TP'] + classification[i]['FP'])
        ) if (classification[i]['TP'] + classification[i]['FP']) != 0 else 0
        recall = (
            classification[i]['TP'] /
            (classification[i]['TP'] + classification[i]['FN'])
        ) if (classification[i]['TP'] + classification[i]['FN']) != 0 else 0
        accuracy = (
            (classification[i]['TP'] + classification[i]['TN']) /
            (classification[i]['TP'] + classification[i]['TN'] +
             classification[i]['FP'] + classification[i]['FN'])
        ) if (classification[i]['TP'] + classification[i]['TN'] +
              classification[i]['FP'] + classification[i]['FN']) != 0 else 0
        classification[i].update({
            'precision': round(precision, 2),
            'recall': round(recall, 2),
            'accuracy': round(accuracy, 2)
        })
        f1_score = (
            2 * (classification[i]['precision'] * classification[i]['recall'])
        ) / (classification[i]['precision'] + classification[i]['recall']) if (
            classification[i]['precision'] +
            classification[i]['recall']) != 0 else 0
        classification[i].update({'f1-score': round(f1_score, 2)})

    num = 0
    denom = 0
    for i in classification:
        num += classification[i]['TP'] + classification[i]['TN']
        denom += classification[i]['TP'] + classification[i][
            'TN'] + classification[i]['FP'] + classification[i]['FN']

    accuracy = num / denom
    classification.update({'accuracy': round(accuracy, 2)})

    print('class | precision | recall | f1-score | accuracy')
    for i in range(max(y_test) + 1):
        print(i, '    | ', classification[i]['precision'], '    | ',
              classification[i]['recall'], ' | ',
              classification[i]['f1-score'], '   | ',
              classification[i]['accuracy'])
    print()
    print('accuracy :    ', classification['accuracy'])
    
def rep_2(df):
    df["rep_1"] = 0
    for scen in range(1, 10):
        start_time = datetime.now()
        for i in df[df.scenarios == scen].index:
            if df.loc[i].repetition_question == 1:
                rep_1 = df.loc[i].task_1
            elif df.loc[i].repetition_question == 2:
                df.loc[i, 'rep_1'] = rep_1
    return df

def rep_3(df):
    df["rep_2"] = 0
    for scen in range(1, 10):
        start_time = datetime.now()
        for i in df[df.scenarios == scen].index:
            if df.loc[i].repetition_question == 2:
                rep_1 = df.loc[i].rep_1
                rep_2 = df.loc[i].task_1
            elif df.loc[i].repetition_question == 3:
                df.loc[i, 'rep_1'] = rep_1
                df.loc[i, 'rep_2'] = rep_2
    return df

In [4]:
#Import the data
data = pd.read_csv(
    "https://ethicallychoice.alwaysdata.net/wp-content/data.csv", sep=";")

#drop the empty line
data = data.dropna()

#convert to int all the datas
for i in ["1", "2"]:
    data["personnage_" + i] = data["personnage_" + i].astype(int)
    data["force_" + i] = data["force_" + i].astype(int)
data["scenarios"] = data["scenarios"].astype(int)

#Import the concentration question
conc = pd.read_csv(
    "https://ethicallychoice.alwaysdata.net/wp-content/conc.csv", sep=";")

#collect people who are bad answering
out = {}
for i in range(len(conc)):
    for j in range(1, 4):
        if conc["q" + str(j)][i] != conc["q" + str(j) + "_rep"][i]:
            if conc.id[i] not in out:
                out[conc.id[i]] = 1
            else:
                out[conc.id[i]] = out[conc.id[i]] + 1
out = [k for (k, v) in out.items() if v > 1]

#collect lines corresponding in data
index = []
for i in range(len(out)):
    index.append(data.index[data["id_participant"] == out[i]].tolist())

#drop these lines
for i in range(len(index)):
    for j in range(len(index[i])):
        data.drop(index[i][j], inplace=True)


#Convert to number "Enfant"
data["enfant"] = [0 if i == "Non" else 1 for i in data["enfant"]]

#Associate a number to each gender
data["sexe"] = [
    0 if i == "Femme" else 1 if i == "Homme" else 2 for i in data["sexe"]
]

#We make the assumption that changing the order does not influence people
data2 = data.copy(deep=True)

for s in ["personnage", "force", "task"]:
    data2[s + "_1"] = data[s + "_2"]
    data2[s + "_2"] = data[s + "_1"]

data = pd.concat([data, data2]).reset_index(drop=True)

data

,personnage_1,personnage_2,force_1,force_2,scenarios,task_1,task_2,repetition_question,id_participant,age,sexe,taille,enfant
0,1,5,7,3,4,8,2,1,d60c0832fc30e645ca04f074c44b49eb,57,0,169,1
1,5,9,3,6,2,1,9,1,d60c0832fc30e645ca04f074c44b49eb,57,0,169,1
2,5,8,3,8,3,1,9,1,d60c0832fc30e645ca04f074c44b49eb,57,0,169,1
3,3,5,10,3,9,0,10,1,d60c0832fc30e645ca04f074c44b49eb,57,0,169,1
4,8,9,8,6,7,5,5,1,d60c0832fc30e645ca04f074c44b49eb,57,0,169,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6745,9,5,6,4,9,6,4,3,bbae47f9f9a4f0cb81fac6cc22f59da6,22,1,165,0
6746,8,4,8,1,6,9,1,3,bbae47f9f9a4f0cb81fac6cc22f59da6,22,1,165,0
6747,8,3,8,10,8,10,0,3,bbae47f9f9a4f0cb81fac6cc22f59da6,22,1,165,0
6748,5,4,4,1,4,7,3,3,bbae47f9f9a4f0cb81fac6cc22f59da6,22,1,165,0


In [5]:
#jeu de données de test pour turing
turing=pd.DataFrame()
for id in data.id_participant.sample(n=5,random_state=1) :
    turing = pd.concat([turing, data[data.id_participant==id]])
    data.drop(data[data.id_participant==id].index, inplace=True)

turing=turing.reset_index(drop=True)
data=data.reset_index(drop=True)

In [6]:
#préparation des jeu de données
data_sample = data.copy()

data_rep2 = rep_2(data)
data_rep3 = rep_3(data_rep2)

data_sample = data_rep3.sample(frac=1, random_state=1)


# RF rep 1

In [7]:
feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe"
]
#We get X and y
Y = data_sample[data_sample.repetition_question == 1]["task_1"]
X = data_sample[data_sample.repetition_question == 1][feature_names]

#We obtain train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.20,
                                                    random_state=42)


In [8]:

RFC = RandomForestClassifier(max_depth=10, max_leaf_nodes=95)

# Number of trees in random forest
n_estimators = [(i + 1) * 10 for i in range(10)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

#We add the other parameters for Random Forest
params = {
    "criterion": ["gini", "entropy","log_loss"],
    'min_samples_split': [2, 3, 4, 8, 10, 16, 32],
    "n_estimators": n_estimators,
    "bootstrap": bootstrap
}

grid_search_cv = GridSearchCV(RFC, params, verbose=1, cv=5, scoring='accuracy', return_train_score=True)
grid_search_cv.fit(X_train, Y_train)
Best_RFC = grid_search_cv.best_estimator_

print("Best score:", grid_search_cv.best_score_)

#saving the model
filename = 'RF1F.pkl'
print("Sauvegarde du modèle dans ", filename)
pickle.dump(Best_RFC, open(filename, "wb"))

# loading the model
RF1F = pickle.load(open(filename, 'rb'))

# printing the model
print(RF1F)

#Test on the test set

Y_test_predict_proba = RF1F.predict_proba(X_test)
Y_test_predict = RF1F.predict(X_test)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_test, Y_test_predict)[0])

print(
    classification_report(Y_test,
                          Y_test_predict,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

Y_test = pd.concat([Y_test], ignore_index=True)

classification_report_opti(Y_test, Y_test_predict)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
Best score: 0.4302235179786201
Sauvegarde du modèle dans  RF1F.pkl
RandomForestClassifier(max_depth=10, max_leaf_nodes=95, min_samples_split=10,
                       n_estimators=80)
Confusion matrix
 [[364  29]
 [ 14  22]]
              precision    recall  f1-score   support

           0       0.43      0.61      0.51        36
           1       0.17      0.07      0.10        15
           2       0.22      0.28      0.24        29
           3       0.39      0.22      0.28        41
           4       0.22      0.16      0.18        38
           5       0.52      0.69      0.59       110
           6       0.41      0.32      0.36        34
           7       0.53      0.25      0.34        40
           8       0.28      0.40      0.33        30
           9       0.33      0.12      0.17        17
          10       0.58      0.64      0.61        39

    accuracy                           0.42       429
   mac

# RF rep 2

In [9]:

feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe","rep_1"
]
#We get X and y
Y = data_sample[data_sample.repetition_question == 2]["task_1"]
X = data_sample[data_sample.repetition_question == 2][feature_names]

#We obtain train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.20,
                                                    random_state=42)



In [10]:
#We get the best parameters for the DT
RFC = RandomForestClassifier(max_depth=10, max_leaf_nodes=95)

# Number of trees in random forest
n_estimators = [(i + 1) * 10 for i in range(10)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

#We add the other parameters for Random Forest
params = {
    "criterion": ["gini", "entropy","log_loss"],
    'min_samples_split': [2, 3, 4, 8, 10, 16, 32],
    "n_estimators": n_estimators,
    "bootstrap": bootstrap
}

grid_search_cv = GridSearchCV(RFC, params, verbose=1, cv=5, scoring='accuracy', return_train_score=True)
grid_search_cv.fit(X_train, Y_train)
Best_RFC = grid_search_cv.best_estimator_

print("Best score:", grid_search_cv.best_score_)

#saving the model
filename = 'RF2F.pkl'
print("Sauvegarde du modèle dans ", filename)
pickle.dump(Best_RFC, open(filename, "wb"))

# loading the model
RF2F = pickle.load(open(filename, 'rb'))

# printing the model
print(RF2F)

#Test on the test set

Y_test_predict_proba = RF2F.predict_proba(X_test)
Y_test_predict = RF2F.predict(X_test)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_test, Y_test_predict)[0])

print(
    classification_report(Y_test,
                          Y_test_predict,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

Y_test = pd.concat([Y_test], ignore_index=True)

classification_report_opti(Y_test, Y_test_predict)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
Best score: 0.6654697969413329
Sauvegarde du modèle dans  RF2F.pkl
RandomForestClassifier(bootstrap=False, max_depth=10, max_leaf_nodes=95,
                       n_estimators=80)
Confusion matrix
 [[395   9]
 [  2  23]]
              precision    recall  f1-score   support

           0       0.72      0.92      0.81        25
           1       0.60      0.27      0.37        22
           2       0.45      0.35      0.39        37
           3       0.50      0.57      0.53        47
           4       0.57      0.65      0.61        26
           5       0.77      0.91      0.83       112
           6       0.57      0.45      0.50        29
           7       0.45      0.48      0.47        42
           8       0.44      0.27      0.33        30
           9       0.53      0.41      0.46        22
          10       0.87      0.92      0.89        37

    accuracy                           0.63       429
   macro av

# RF rep 3

In [11]:


feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe","rep_1","rep_2"
]
#We get X and y
Y = data_sample[data_sample.repetition_question == 3]["task_1"]
X = data_sample[data_sample.repetition_question == 3][feature_names]

#We obtain train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.20,
                                                    random_state=42)



In [12]:
#We get the best parameters for the DT
RFC = RandomForestClassifier(max_depth=10, max_leaf_nodes=95)

# Number of trees in random forest
n_estimators = [(i + 1) * 10 for i in range(10)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

#We add the other parameters for Random Forest
params = {
    "criterion": ["gini", "entropy","log_loss"],
    'min_samples_split': [2, 3, 4, 8, 10, 16, 32],
    "n_estimators": n_estimators,
    "bootstrap": bootstrap
}

grid_search_cv = GridSearchCV(RFC, params, verbose=1, cv=5, scoring='accuracy', return_train_score=True)
grid_search_cv.fit(X_train, Y_train)
Best_RFC = grid_search_cv.best_estimator_

print("Best score:", grid_search_cv.best_score_)

#saving the model
filename = 'RF3F.pkl'
print("Sauvegarde du modèle dans ", filename)
pickle.dump(Best_RFC, open(filename, "wb"))

# loading the model
RF3F = pickle.load(open(filename, 'rb'))

# printing the model
print(RF3F)

#Test on the test set

Y_test_predict_proba = RF3F.predict_proba(X_test)
Y_test_predict = RF3F.predict(X_test)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_test, Y_test_predict)[0])

print(
    classification_report(Y_test,
                          Y_test_predict,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

Y_test = pd.concat([Y_test], ignore_index=True)

classification_report_opti(Y_test, Y_test_predict)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
Best score: 0.7635926551071558
Sauvegarde du modèle dans  RF3F.pkl
RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=10,
                       max_leaf_nodes=95, min_samples_split=10,
                       n_estimators=70)
Confusion matrix
 [[380   4]
 [  7  38]]
              precision    recall  f1-score   support

           0       0.90      0.84      0.87        45
           1       0.67      0.67      0.67        21
           2       0.65      0.61      0.62        33
           3       0.66      0.76      0.71        38
           4       0.74      0.70      0.72        37
           5       0.84      0.84      0.84       103
           6       0.76      0.68      0.72        38
           7       0.72      0.71      0.72        48
           8       0.46      0.65      0.54        17
           9       0.75      0.75      0.75        20
          10       0.86      0.83      0.84        29



# Turing

In [13]:
#jeu de données de test pour turing
turing=rep_2(turing)
turing=rep_3(turing)
turing.sample(frac=1, random_state=1)
turing=turing.reset_index(drop=True)

In [14]:
human=turing.id_participant.sample(n=3,random_state=1)
human=human.reset_index(drop=True)
human=pd.concat([human,pd.Series([1,7,4]),pd.Series([8,5,2]),pd.Series([6,3,9])],keys=['id','scen_1','scen_2','scen_3'],axis=1)

human

,id,scen_1,scen_2,scen_3
0,22d27f6173cf259249d7d5e672e5664f,1,8,6
1,c040a51377c5c9810a5e21e83971e341,7,5,3
2,dd23df5f25fb2015df462104eff67770,4,2,9


In [15]:
robot=turing.id_participant.sample(n=3,random_state=0)
robot=robot.reset_index(drop=True)
robot=pd.concat([robot,pd.Series([2,8,5]),pd.Series([9,6,3]),pd.Series([4,1,7])],keys=['id','scen_1','scen_2','scen_3'],axis=1)

robot

,id,scen_1,scen_2,scen_3
0,c040a51377c5c9810a5e21e83971e341,2,9,4
1,d00d295a4103ea0be6024e174efd6465,8,6,1
2,490895544f1107af8fccf3d3930e60a8,5,3,7


## rep1

In [16]:
feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe"
]
#We get X and y
Y_turing = turing[turing.repetition_question == 1]["task_1"]
X_turing = turing[turing.repetition_question == 1][feature_names]

y_test_predict_proba = RF1F.predict_proba(X_turing)
y_test_predict_turing = RF1F.predict(X_turing)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_turing, y_test_predict_turing)[0])

print(
    classification_report(Y_turing,
                          y_test_predict_turing,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

y_turing = pd.concat([Y_turing],ignore_index=True)

classification_report_opti(y_turing, y_test_predict_turing)

y_pred = pd.Series(y_test_predict_turing,
                   index=turing[turing.repetition_question == 1].index,
                   name='task_1 prédite')
result_rep1 = pd.concat([y_pred, Y_turing, turing[turing.repetition_question == 1]], axis=1)

Confusion matrix
 [[96  8]
 [ 2  2]]
              precision    recall  f1-score   support

           0       0.20      0.50      0.29         4
           1       0.33      0.20      0.25        10
           2       0.44      0.44      0.44         9
           3       0.00      0.00      0.00         9
           4       0.33      0.11      0.17         9
           5       0.45      0.81      0.58        26
           6       0.50      0.11      0.18         9
           7       0.00      0.00      0.00         9
           8       0.38      0.33      0.35         9
           9       0.67      0.20      0.31        10
          10       0.30      0.75      0.43         4

    accuracy                           0.36       108
   macro avg       0.33      0.31      0.27       108
weighted avg       0.36      0.36      0.31       108

class | precision | recall | f1-score | accuracy
0     |  0.25     |  1.0  |  0.4    |  0.89
1     |  0.28     |  0.7  |  0.4    |  0.81
2     |  0.35

In [17]:
infos=[ "scenarios","task_1 prédite","personnage_1","personnage_2","force_1", "force_2", "repetition_question"]
for h in robot.index:
    print(robot.loc[h].id)
    display(result_rep1[infos][result_rep1.id_participant==robot.loc[h].id][result_rep1.scenarios==robot.loc[h].scen_1].drop_duplicates(subset=['scenarios'], keep='first'))
    

c040a51377c5c9810a5e21e83971e341


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
220,2,2,1,9,3,4,1


d00d295a4103ea0be6024e174efd6465


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
54,8,0,3,8,10,8,1


490895544f1107af8fccf3d3930e60a8


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
2,5,0,2,3,4,10,1


In [18]:
infos=[ "scenarios","task_1","personnage_1","personnage_2","force_1", "force_2", "repetition_question"]
for h in human.index:
    print(human.loc[h].id)
    display(result_rep1[infos][result_rep1.id_participant==human.loc[h].id][result_rep1.scenarios==human.loc[h].scen_1].drop_duplicates(subset=['scenarios'], keep='first'))
    

22d27f6173cf259249d7d5e672e5664f


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,repetition_question
108,1,8,8,2,4,4,1,1


c040a51377c5c9810a5e21e83971e341


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,repetition_question
221,7,8,8,1,9,3,4,1


dd23df5f25fb2015df462104eff67770


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,repetition_question
166,4,0,0,4,8,1,12,1


## rep2

In [19]:
feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe","rep_1"
]
#We get X and y
Y_turing = turing[turing.repetition_question == 2]["task_1"]
X_turing = turing[turing.repetition_question == 2][feature_names]

y_test_predict_proba = RF2F.predict_proba(X_turing)
y_test_predict_turing = RF2F.predict(X_turing)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_turing, y_test_predict_turing)[0])

print(
    classification_report(Y_turing,
                          y_test_predict_turing,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

y_turing = pd.concat([Y_turing],ignore_index=True)

classification_report_opti(y_turing, y_test_predict_turing)

y_pred = pd.Series(y_test_predict_turing,
                   index=turing[turing.repetition_question == 2].index,
                   name='task_1 prédite')

result_rep2 = pd.concat([y_pred, Y_turing, turing[turing.repetition_question == 2]], axis=1)


Confusion matrix
 [[100   1]
 [  4   3]]
              precision    recall  f1-score   support

           0       0.75      0.43      0.55         7
           1       0.60      0.33      0.43         9
           2       0.45      0.56      0.50         9
           3       0.14      0.40      0.21         5
           4       0.67      0.36      0.47        11
           5       0.79      0.85      0.81        26
           6       0.75      0.55      0.63        11
           7       0.12      0.40      0.19         5
           8       0.33      0.11      0.17         9
           9       0.44      0.44      0.44         9
          10       0.75      0.43      0.55         7

    accuracy                           0.51       108
   macro avg       0.53      0.44      0.45       108
weighted avg       0.60      0.51      0.53       108

class | precision | recall | f1-score | accuracy
0     |  0.44     |  0.57  |  0.5    |  0.93
1     |  0.35     |  0.78  |  0.48    |  0.86
2     

In [20]:
infos=[ "scenarios","task_1 prédite","personnage_1","personnage_2","force_1", "force_2", "repetition_question"]
for h in robot.index:
    print(robot.loc[h].id)
    display(result_rep2[infos][result_rep2.id_participant==robot.loc[h].id][result_rep2.scenarios==robot.loc[h].scen_2].drop_duplicates(subset=['scenarios'], keep='first'))
    display(result_rep1[infos][result_rep1.id_participant==robot.loc[h].id][result_rep1.scenarios==robot.loc[h].scen_2].drop_duplicates(subset=['scenarios'], keep='first'))


c040a51377c5c9810a5e21e83971e341


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
227,9,6,2,8,2,5,2


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
222,9,5,2,8,2,5,1


d00d295a4103ea0be6024e174efd6465


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
64,6,6,8,9,8,7,2


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
59,6,6,8,9,8,7,1


490895544f1107af8fccf3d3930e60a8


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
11,3,3,4,9,1,6,2


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
4,3,2,4,9,1,6,1


In [21]:
infos=[ "scenarios","task_1","personnage_1","personnage_2","force_1", "force_2","rep_1", "repetition_question"]
for h in human.index:
    print(human.loc[h].id)
    display(result_rep2[infos][result_rep2.id_participant==human.loc[h].id][result_rep2.scenarios==human.loc[h].scen_2].drop_duplicates(subset=['scenarios'], keep='first'))
    

22d27f6173cf259249d7d5e672e5664f


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,repetition_question
124,8,5,5,2,4,4,1,5,2


c040a51377c5c9810a5e21e83971e341


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,repetition_question
230,5,3,3,2,9,2,4,4,2


dd23df5f25fb2015df462104eff67770


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,repetition_question
171,2,4,4,1,2,8,6,5,2


## rep3

In [22]:
feature_names = [
    "force_1", "force_2", "scenarios", "age", "sexe","rep_1","rep_2"
]
#We get X and y
Y_turing = turing[turing.repetition_question == 3]["task_1"]
X_turing = turing[turing.repetition_question == 3][feature_names]

y_test_predict_proba = RF3F.predict_proba(X_turing)
y_test_predict_turing = RF3F.predict(X_turing)

print("Confusion matrix\n",
      multilabel_confusion_matrix(Y_turing, y_test_predict_turing)[0])

print(
    classification_report(Y_turing,
                          y_test_predict_turing,
                          output_dict=False,
                          target_names=[str(i) for i in range(11)]))

y_turing = pd.concat([Y_turing],ignore_index=True)

classification_report_opti(y_turing, y_test_predict_turing)

y_pred = pd.Series(y_test_predict_turing,
                   index=turing[turing.repetition_question == 3].index,
                   name='task_1 prédite')

result_rep3 = pd.concat([y_pred, Y_turing, turing[turing.repetition_question == 3]], axis=1)


Confusion matrix
 [[98  1]
 [ 1  8]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.86      0.75      0.80         8
           2       0.38      0.60      0.46         5
           3       0.33      0.22      0.27         9
           4       0.50      0.62      0.56         8
           5       0.90      0.87      0.88        30
           6       0.62      0.62      0.62         8
           7       0.43      0.33      0.38         9
           8       0.44      0.80      0.57         5
           9       0.75      0.75      0.75         8
          10       1.00      0.78      0.88         9

    accuracy                           0.69       108
   macro avg       0.65      0.66      0.64       108
weighted avg       0.71      0.69      0.70       108

class | precision | recall | f1-score | accuracy
0     |  0.5     |  0.89  |  0.64    |  0.92
1     |  0.29     |  0.88  |  0.44    |  0.83
2     |  0

In [23]:
infos=[ "scenarios","task_1 prédite","personnage_1","personnage_2","force_1", "force_2", "repetition_question"]
for h in robot.index:
    print(robot.loc[h].id)
    display(result_rep3[infos][result_rep3.id_participant==robot.loc[h].id][result_rep3.scenarios==robot.loc[h].scen_3].drop_duplicates(subset=['scenarios'], keep='first'))
    display(result_rep2[infos][result_rep2.id_participant==robot.loc[h].id][result_rep2.scenarios==robot.loc[h].scen_3].drop_duplicates(subset=['scenarios'], keep='first'))
    display(result_rep1[infos][result_rep1.id_participant==robot.loc[h].id][result_rep1.scenarios==robot.loc[h].scen_3].drop_duplicates(subset=['scenarios'], keep='first'))


c040a51377c5c9810a5e21e83971e341


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
242,4,7,4,5,1,2,3


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
225,4,7,4,5,1,2,2


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
218,4,5,4,5,1,2,1


d00d295a4103ea0be6024e174efd6465


...

,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
62,1,5,1,8,5,8,1


490895544f1107af8fccf3d3930e60a8


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
23,7,5,8,9,6,6,3


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
14,7,5,8,9,6,6,2


,scenarios,task_1 prédite,personnage_1,personnage_2,force_1,force_2,repetition_question
8,7,5,8,9,6,6,1


In [24]:
infos=[ "scenarios","task_1","personnage_1","personnage_2","force_1", "force_2","rep_1","rep_2", "repetition_question"]
for h in human.index:
    print(human.loc[h].id)
    display(result_rep3[infos][result_rep3.id_participant==human.loc[h].id][result_rep3.scenarios==human.loc[h].scen_3].drop_duplicates(subset=['scenarios'], keep='first'))
    

22d27f6173cf259249d7d5e672e5664f


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,rep_2,repetition_question
131,6,1,1,4,9,1,6,1,1,3


c040a51377c5c9810a5e21e83971e341


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,rep_2,repetition_question
235,3,9,9,3,9,10,4,9,7,3


dd23df5f25fb2015df462104eff67770


,scenarios,task_1,task_1,personnage_1,personnage_2,force_1,force_2,rep_1,rep_2,repetition_question
183,9,2,2,3,9,10,10,3,2,3


In [25]:
import random
array=[1,2,3,1,2,3,1,2,3,1,2,3,1,2,3]
random.shuffle(array)
array


[2, 3, 3, 1, 2, 3, 3, 2, 1, 1, 2, 3, 2, 1, 1]

In [26]:
array=[0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
random.shuffle(array)
array


[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]